In [1]:
import json
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('C://Users//LENOVO//projects//Health Care Chatbot//data//intents.json').read()
intents = json.loads(data_file)

In [2]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello, thanks for asking',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'context': ['']},
  {'tag': 'options',
   'patterns': ['How you could help me?',
    'What you can do

In [3]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
classes

['greeting',
 'goodbye',
 'thanks',
 'options',
 'tell_symptoms',
 'tell_precautions',
 'tell_description']

In [6]:
import pickle

In [7]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

33 documents
7 classes ['goodbye', 'greeting', 'options', 'tell_description', 'tell_precautions', 'tell_symptoms', 'thanks']
62 unique lemmatized words ["'s", ',', 'about', 'am', 'anyone', 'are', 'awesome', 'be', 'bye', 'can', 'chatting', 'could', 'day', 'description', 'disease', 'do', 'feeling', 'fine', 'for', 'from', 'give', 'good', 'goodbye', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'is', 'it', 'later', 'me', 'more', 'next', 'nice', 'not', 'offered', 'please', 'precaution', 'provide', 'see', 'should', 'spreading', 'stop', 'support', 'take', 'tell', 'thank', 'thanks', 'that', 'there', 'this', 'till', 'time', 'to', 'unusual', 'what', 'you']


In [8]:
print(documents)

[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['How', 'you', 'could', 'help', 'me', '?'], 'options'), (['What', 'you', 'can', 'do', '?'], 'options'), (['What', 'help', 'you', 'provide', '?'], 'options'), (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'), (['What', 'support', 'is', 'offered', '?'], 'options'), (['I', 'am', 'not', 'feeling', 'fine'], 'tell_symptoms'), (['I', 'am', 'not', 'feeling', 'good'], 'tell_symptoms'), (['I', 'am', 'feeling', 'un

In [9]:
import random
import numpy as np
training = []
output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
random.shuffle(training)
training = np.array(training)
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


c:\users\lenovo\anaconda3\envs\python37\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [10]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

In [11]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
7/7 [==============================] - 1s 5ms/step - loss: 2.0561 - accuracy: 0.0431
Epoch 2/200
7/7 [==============================] - 0s 2ms/step - loss: 1.9371 - accuracy: 0.1579
Epoch 3/200
7/7 [==============================] - 0s 2ms/step - loss: 1.8974 - accuracy: 0.2700
Epoch 4/200
7/7 [==============================] - 0s 2ms/step - loss: 1.8347 - accuracy: 0.3429
Epoch 5/200
7/7 [==============================] - 0s 2ms/step - loss: 1.6584 - accuracy: 0.5763
Epoch 6/200
7/7 [==============================] - 0s 2ms/step - loss: 1.7515 - accuracy: 0.3506
Epoch 7/200
7/7 [==============================] - 0s 2ms/step - loss: 1.6273 - accuracy: 0.4567
Epoch 8/200
7/7 [==============================] - 0s 2ms/step - loss: 1.4724 - accuracy: 0.6048
Epoch 9/200
7/7 [==============================] - 0s 2ms/step - loss: 1.4321 - accuracy: 0.4889
Epoch 10/200
7/7 [==============================] - 0s 2ms/step - loss: 1.4514 - accuracy: 0.6570
Epoch 11/200
7/7 [===========

7/7 [==============================] - 0s 2ms/step - loss: 0.0324 - accuracy: 1.0000
Epoch 85/200
7/7 [==============================] - 0s 2ms/step - loss: 0.0453 - accuracy: 0.9924
Epoch 86/200
7/7 [==============================] - 0s 2ms/step - loss: 0.0317 - accuracy: 1.0000
Epoch 87/200
7/7 [==============================] - 0s 2ms/step - loss: 0.0838 - accuracy: 0.9833
Epoch 88/200
7/7 [==============================] - 0s 2ms/step - loss: 0.0579 - accuracy: 0.9687
Epoch 89/200
7/7 [==============================] - 0s 2ms/step - loss: 0.0604 - accuracy: 1.0000
Epoch 90/200
7/7 [==============================] - 0s 2ms/step - loss: 0.0573 - accuracy: 0.9924
Epoch 91/200
7/7 [==============================] - 0s 2ms/step - loss: 0.0807 - accuracy: 1.0000
Epoch 92/200
7/7 [==============================] - 0s 2ms/step - loss: 0.0405 - accuracy: 1.0000
Epoch 93/200
7/7 [==============================] - 0s 2ms/step - loss: 0.1604 - accuracy: 0.9444
Epoch 94/200
7/7 [===============

7/7 [==============================] - 0s 2ms/step - loss: 0.0962 - accuracy: 0.9687
Epoch 167/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0183 - accuracy: 1.0000
Epoch 168/200
7/7 [==============================] - 0s 2ms/step - loss: 0.0449 - accuracy: 1.0000
Epoch 169/200
7/7 [==============================] - 0s 2ms/step - loss: 0.0254 - accuracy: 1.0000
Epoch 170/200
7/7 [==============================] - 0s 2ms/step - loss: 0.1007 - accuracy: 0.9883
Epoch 171/200
7/7 [==============================] - 0s 2ms/step - loss: 0.0201 - accuracy: 1.0000
Epoch 172/200
7/7 [==============================] - 0s 2ms/step - loss: 0.0165 - accuracy: 1.0000
Epoch 173/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0150 - accuracy: 1.0000
Epoch 174/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0102 - accuracy: 1.0000
Epoch 175/200
7/7 [==============================] - 0s 2ms/step - loss: 0.0561 - accuracy: 0.9312
Epoch 176/200
7/7 [=====

In [12]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=True)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result,tag

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res,tag = getResponse(ints, intents)
    return res,tag

In [13]:
chatbot_response("hi")

found in bag: hi


('Hi there, how can I help?', 'greeting')

In [14]:
disease_model= pickle.load(open('C://Users//LENOVO//projects//Health Care Chatbot//notebook//Multinomial_classifier_disease.pkl','rb'))
disease_tokenizer = pickle.load(open('C://Users//LENOVO//projects//Health Care Chatbot//notebook//tf_idf_vectorizer_disease.pkl','rb'))

In [15]:

def clean(text):
    text = text.lower() 
    text = text.split()
    text = ' '.join(text)
    return text


In [16]:
while True:
    ip=input("chat with bot")
    if ip=="chal hat":
        break
    else:
        res,tag=chatbot_response(ip)
        if tag=="tell_symptoms":
            print("enter your all symptoms.")
            ip=input("chat with bot")
            symptoms=ip.split(',')
            symptoms=" ".join(symptoms)
#             print(symptoms)
            test=clean(symptoms)
            test=[test]
            test_vectorized=disease_tokenizer.transform(test)
#             print(test_vectorized.toarray())
            pred=disease_model.predict(test_vectorized)
            print(pred)
        print("bot reply :",res)

chat with bothii
bot reply : Hi there, how can I help?
chat with botchal hat
